# 🧠 Web Content Summarizer

This notebook extracts and summarizes text content from websites using OpenAI-compatible APIs (Groq or Ollama).

## Features:
- Scrapes content using `requests` or `Selenium`
- Uses OpenAI-compatible models like `grok` & Local models like `Ollama` to generate summaries
- Supports custom URLs


In [1]:
# !pip install requests python-dotenv beautifulsoup4 ipython openai selenium webdriver-manager


In [2]:
import os
import requests
import json
import socket
import ipaddress
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from IPython.display import display, Markdown, update_display
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

## 🔧 1. Load API Keys
We load the API key from a `.env` file to securely connect to the Groq or Ollama LLM backend.


In [3]:
load_dotenv(override=True)
api_key  = os.getenv("GROQ_API_KEY")

if not api_key:
    print("No API_Key found, Please set the API_KEY.")
    exit(1)
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [4]:
openai = OpenAI(api_key=api_key, base_url = "https://api.groq.com/openai/v1")
ollama_with_openai = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

## 🌐 2. Website Content Extractor
This class retrieves the raw HTML and title of a webpage using `requests` or Selenium.


In [5]:
"""
Explanation of this code: https://chatgpt.com/share/686f8aed-a210-8007-970d-37906975fa4f
"""


def is_safe_url(url):
    try:
        parsed = urlparse(url)
        if parsed.scheme not in ["http", "https"] or parsed.netloc == "":
            return False

        host = parsed.hostname
        ip = ipaddress.ip_address(socket.gethostbyname(host))
        if ip.is_private or ip.is_loopback or ip.is_reserved or ip.is_link_local:
            return False
    except Exception:
        return False
    return True


class Web_Scapper:
    """
    A utility class to represent a Website that we have scraped, using Selenium, with extracted links.
    """

    def __init__(self, url):
        if not is_safe_url(url):
            raise ValueError("Invalid or unsafe URL")

        self.url = url
        self.title = "No title found"
        self.text = ""
        self.links = []

        # Setup Selenium
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-extensions")
        options.add_argument("--disable-blink-features=AutomationControlled")

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        try:
            driver.set_page_load_timeout(60)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Get title
            self.title = soup.title.string.strip() if soup.title else "No title found"

            # Remove irrelevant tags
            if soup.body:
                for tag in soup.body(["script", "style", "img", "input"]):
                    tag.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)

            # Extract all valid links
            all_links = [a.get("href") for a in soup.find_all("a") if a.get("href")]
            self.links = all_links

        except WebDriverException as e:
            print(f"Error loading page with Selenium: {e}")
        finally:
            driver.quit()

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"


In [6]:
# website = Web_Scapper("https://edwarddonner.com")
# # print(web_scrapper.get_contents())
# print(website.get_contents())
# # web_scrapper.links


Webpage Title:
Home - Edward Donner

Webpage Contents:
Skip to content
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has 

## 📝 3. Prompt Engineering and Summarization
This section defines the prompts used to instruct the `large language model (LLM)` for summarization and provides the core functions to perform the website summarization using the `selected LLM (Groq or Ollama)`.


In [7]:
system_prompt = (
    "You are a highly intelligent assistant tasked with analyzing website content. "
    "Your job is to extract and summarize the **core purpose** and **main content** of the site, ignoring any navigation bars, footers, cookie banners, or repetitive UI elements. "
    "You excel at identifying meaningful information such as services offered, articles, announcements, product details, or business descriptions. "
    "Always format your response in **markdown**, and present the summary in a clean, human-readable format that would make sense to someone who has never seen the site before."
)


def user_prompt_for(website):
    user_prompt = (
        f"## Website Title\n"
        f"**{website.title}**\n\n"
        f"## Instructions\n"
        f"You are analyzing the contents of this website. Please provide a clear and concise markdown summary of the website’s purpose and content. "
        f"Include any relevant details such as:\n"
        f"- Services, products, or features offered\n"
        f"- Blog posts, articles, or resources\n"
        f"- News or announcements\n"
        f"- Contact information, locations, or teams (if present)\n\n"
        f"### Important:\n"
        f"- **Ignore** navigation links, UI controls, cookie notices, or footer text.\n"
        f"- Focus on meaningful, unique content visible to a visitor.\n\n"
        f"## Website Content\n"
        f"{website.text}"
    )
    return user_prompt


In [8]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    website = Web_Scapper(url)
    response = openai.chat.completions.create(
        model = "llama3-70b-8192",
        messages = [
        {"role": "system", "content": system_prompt}
        ,{"role": "user", "content": user_prompt_for(website)}
    ]
    )
    return response.choices[0].message.content


# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


In [9]:
display_summary("https://www.microwebtec.com/")

**Microweb Software Pvt Ltd: Full Stack Development Company**
===============================================

**Core Purpose:**
Microweb Software Pvt Ltd is a full-stack development company that specializes in translating digital challenges into seamless strategies, focusing on delivering exceptional end-user experiences. They combine design, technology, and data to create engaging digital platforms.

**Main Content:**

### Services

* Cloud Application Development Services
* Azure DevOps Services
* Golang Development Services
* DevOps Consulting Services
* Business Transformation
* Laravel Application Development
* Symfony Web Development
* Node.js Development
* AngularJs Web Development Services
* Ruby on Rails Application Development
* Microsoft Development
* Mobile Application Development
* IoT and Embedded Systems & Smart Solutions
* Cloud Technology
* ReactJS Development
* Drupal Web Development Services
* 2D and 3D Video Animation
* UI & UX Design
* Web Development
* Enterprise Solutions
* Digital Marketing
* Software Outsource to India
* Graphic Designing
* eCommerce Development
* WordPress Web Development
* WooCommerce Development
* Shopify Development
* Python Development
* AI & ML Services

### Case Studies

* Crefin – Process Automation for Loan Processing in the USA
* SkyEduco – AI-Powered Study Abroad Portal with AI Features
* Idiagnose – AI-Powered Chatbot to Train Doctors

### Contact Information

* Location: 502, Matrix Building, Corporate Road, Prahladnagar, Behind Divya Bhaskar, Makarba, Ahmedabad-380015, India
* Phone: +91 94264 83914
* Email: [available on the website]

Note: The website does not have any blog posts, articles, or resources apart from the case studies.

In [10]:
display_summary("https://www.gopalinfo.com/")

**Gopal Info Website Summary**
=============================

**Core Purpose**
Gopal Info is a digital agency providing expert solutions in graphic design, website development, digital marketing, SEO, and social media management to help businesses thrive online.

**Main Content**
### Services

* Graphic Designing: Crafting eye-catching designs to elevate brands
* UI/UX Design: Creating intuitive and engaging user experiences
* Branding Consulting: Building strong, memorable brands with expert guidance
* Social Media Marketing: Driving engagement and growth with targeted strategies
* Search Engine Optimization: Boosting online visibility with proven SEO strategies
* Google Ads: Maximizing reach and conversions with targeted campaigns
* Web Designing: Creating stunning websites that capture brand essence
* Website Development: Building robust and scalable web solutions
* Mobile App Development: Developing innovative mobile apps that enhance user experience

### Technologies
Gopal Info utilizes a range of advanced technologies to deliver high-quality services.

### How we work
The agency follows an agile methodology prioritizing collaboration, creativity, and quality at every step, from initial consultation to final delivery.

### Testimonials
Empowering businesses with innovative digital solutions to drive growth and success.

### Contact Information
Address: 502 - Matrix, Makarba, Ahmedabad - 380015, India
Get in touch through the website's contact form or WhatsApp.

In [11]:
# display_summary("https://techcrunch.com")